# Median combination of GMOS-N and GMOS-S flatfield files

Creates median-combined flatfield images (as individual extension files) for processing GMOS-N and GMOS-S science data after initial reduction (overscan correction, trimming, and bias subtraction) and screening to ensure mean counts for all files lie within a desired range.

## Before running notebook:

- Install <tt>fpack</tt> and <tt>funpack</tt> if not already installed (https://heasarc.gsfc.nasa.gov/fitsio/fpack/); change paths to local installation location in functions <tt>compress_file_fpack</tt> and <tt>decompress_file_fpack</tt> in this notebook<br><br>

- Run <tt>notebook_GMOS_a01_12amp_process_raw_twilight_flats.ipynb</tt> to perform initial processing of bias and flatfield images for the given night<br><br>

- Create folder named "<tt>exclude</tt>" in <tt>procfits_twiskyflat</tt> folder in data directory for the given night produced by the previous notebook.  The existence of this folder is used by this notebook to determine whether a given data folder is ready for further processing<br><br>

- View image statistics file ("<tt>nYYYYMMDD.stats</tt>") in <tt>procfits_twiskyflat</tt> folder in data directory for the given night, and determine which twilight sky frames, if any, should be excluded from construction of a median flatfield (i.e., has mean counts that are too low or too high, e.g., < 10000 or >50000); move these frames (make sure to move all three chips associated with each frame) to <tt>exclude</tt> folder to remove from further processing<br><br>

- Run <tt>process_flatfield_directory(base_path,date_id,filter_id,observatory)</tt> where base_path is the directory containing the <tt>procfits_twiskyflat</tt> sub-directory, <tt>date_id</tt> is the date of observations in YYYYMMDD format, <tt>filter_id</tt> is the one-letter name of the filter used for the flatfield images (e.g., <tt>g</tt>, <tt>r</tt>, <tt>i</tt>, or <tt>z</tt>), and observatory is either "<tt>GeminiN</tt>" or "<tt>GeminiS</tt>"


In [ ]:
%matplotlib inline
import os, sys
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData
import numpy as np
import glob, subprocess
import datetime


In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def compress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/fpack',filename])
    os.remove(filename)
    return None

def decompress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/funpack',filename])
    os.remove(filename)
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None


In [ ]:
def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    for raw_mef_file in glob.glob('*.fits'):
        fits_date = raw_mef_file[1:9]
    return fits_date

def flatfield_median_combine(flatfield_name,reduction_log):
    print('{:s} - Starting median combination of flatfield frames...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Starting median combination of flatfield frames...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip1.fits'
    for fits_file in glob.glob('*.chip1.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data)
            fits_data_n = fits_data / fits_mean
            #file_prefix = fits_file[0:19]
            #fits_file_n = file_prefix + '_n.otz.fits'
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 1 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 1 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip2.fits'
    for fits_file in glob.glob('*.chip2.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data[251:1012,1:2042])
            fits_data_n = fits_data / fits_mean
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 2 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 2 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip3.fits'
    for fits_file in glob.glob('*.chip3.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data)
            fits_data_n = fits_data / fits_mean
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 3 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 3 complete.\n')

    print('{:s} - Median combination of flatfield frames complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Median combination of flatfield frames complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    return None


In [ ]:
def process_flatfield_directory(data_path,date_id,filter_id,observatory):
    instrname = 'not recognized'
    if observatory == 'GeminiN': instrname = 'gmosn'
    if observatory == 'GeminiS': instrname = 'gmoss'

    os.chdir(data_path)
    path_rawfits  = data_path + 'rawfits_twiskyflat/'
    path_rawbias  = data_path + 'rawfits_bias/'
    path_procfits = data_path + 'procfits_twiskyflat/'
    path_exclfits = data_path + 'procfits_twiskyflat/exclude/'
    
    flatfield_name = 'n' + date_id + '.' + instrname + '.twiskyflat.' + filter_id
    flatfits_file_chip2 = path_procfits + flatfield_name + '.chip2.fits.fz'
    reduction_logfile = data_path + 'log_reduction_' + date_id + '_' + instrname + '.txt'
    
    if os.path.exists(flatfits_file_chip2):
        print('{:s} - Median flatfields for {:s} already generated.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
    elif not os.path.exists(path_procfits):
        print('{:s} - Flatfield files in {:s} not yet processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
    elif not os.path.exists(path_exclfits):
        print('{:s} - Flatfield files in {:s} not yet screened.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
    elif instrname == 'not recognized':
        print('{:s} - Instrument for {:s} not recognized.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
    else:
        with open(reduction_logfile,'w') as reduction_log:
            print('{:s} - Starting median combination of flatfield files in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            reduction_log.write('{:s} - Starting median combination of flatfield files in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            os.chdir(path_procfits)
            decompress_directory_fpack(path_procfits)                
            flatfield_median_combine(flatfield_name,reduction_log)
            compress_directory_fpack(path_procfits)     
            print('{:s} - Processing flatfield files in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
            reduction_log.write('{:s} - Processing flatfield files in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_path))
        
    return None


In [ ]:
# date_id: date of observations in YYYYMMDD format
# filter_id: single letter filter name (e.g., g, r, i, or z)
# observatory: 'GeminiN' or 'GeminiS'

process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/data_calib/GMOS-S_TwilightFlats_20150101_present/GMOS-S_Twilight_Flats_r/test_obs/','20150115','r','GeminiS')
